In [2]:
!pip install -q -U watermark

In [3]:
#1 Imports
import re
import pickle
import nltk
import sklearn
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score

In [4]:
#2 Versions of the packages used in this Jupyter notebook
%reload_ext watermark
%watermark -a "panData"

Author: panData



In [5]:
#load the dataset
df = pd.read_csv(r"C:\Users\Dhanashree\Downloads\IMDB Dataset.csv")

In [6]:
#Shape
df.shape

(50000, 2)

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [9]:
# Count of records by class
df.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

We are working with text data.We cannot perform mathematicsl operation directly on the text data. So? Conert the data into numerical representation

In [10]:
#Adjust the labels for numerical representation
#we used replace function to replace the string value into numerical value
df.sentiment.replace('positive', 1, inplace = True)
df.sentiment.replace('negative', 0, inplace = True)
#We use inplace because it saves rhis substitution directly in the data frame

In [11]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


 <br /><br / This are the html tags which indicates it was extracted froma html web page.
They are not neccessary.
So What we do is... Create a function for this using re, which is regualar expressions.

In [12]:
#General data cleaning function 
def clean_data(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned, '', text)

In [13]:
#Testing the function
text_with_tags = "<p>This is an example <b> with </b> HTML tags.</p>"
cleaned_text = clean_data(text_with_tags)
print(cleaned_text)

This is an example  with  HTML tags.


In [14]:
#Apply the function to our dataset
df.review = df.review.apply(clean_data)

In [15]:
#View the first review
df.review[0]


"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [16]:
# Function for cleaning special charecters
def clean_special_charecters(text):
    rem = '' #empty string used to store resultting text
    for i in text: #created a for loop 
        if i.isalnum(): # if the word is alpha then it takes that word
           rem = rem + i
            
        else:
           rem = rem + '' #otherwise it discards that word

    return rem

In [17]:
#Testing the function"
test_with_special_characters = "Hello!!!, world!!"
cleaned_text = clean_special_charecters(test_with_special_characters)
print(cleaned_text)

Helloworld


In [18]:
# Apply the function
df_review = df.review.apply(clean_special_charecters)

#View the first review
df.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [19]:
#function to covert text to lower
def convert_to_lowercase(text):
    return text.lower()

In [20]:
#Testin gthe function
sentence = "This SENTENCE with LowercAse"
output_sentence = convert_to_lowercase(sentence)
print(output_sentence)

this sentence with lowercase


In [21]:
#Apply the function
df.review = df.review.apply(convert_to_lowercase)

#View the first review
df.review[0]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [22]:
#Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dhanashree\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dhanashree\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stopwords(text):
    stop_words = set(stopwords.words('english')) # stop words from english language
    words = word_tokenize(text) # splits the texts into words based on spaces and puntuation
    return [word for word in words if word.lower() not in stop_words]

In [24]:
#Testing the function
sentence = "They are right,as this is exactly what happened with me"
output_sentence = remove_stopwords(sentence)
print(output_sentence)

['right', ',', 'exactly', 'happened']


In [25]:
pip install ipython


Note: you may need to restart the kernel to use updated packages.


In [26]:
#Measure the time of the excuetion
'''%%time
df.review = df.review.apply(remove_stopwords)'''

'%%time\ndf.review = df.review.apply(remove_stopwords)'

In [27]:
%%time
df['review'] = df['review'].apply(remove_stopwords)

CPU times: total: 2min 48s
Wall time: 3min 32s


In [28]:
# Functions for stemming
def stemmer(text):
    stemmer_object = SnowballStemmer('english') #that creates object to help us prefixes and suffixes
    return " ".join([stemmer_object.stem(w) for w in text])

In [29]:
df.review[0]

['one',
 'reviewers',
 'mentioned',
 'watching',
 '1',
 'oz',
 'episode',
 "'ll",
 'hooked',
 '.',
 'right',
 ',',
 'exactly',
 'happened',
 'me.the',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scenes',
 'violence',
 ',',
 'set',
 'right',
 'word',
 'go',
 '.',
 'trust',
 ',',
 'show',
 'faint',
 'hearted',
 'timid',
 '.',
 'show',
 'pulls',
 'punches',
 'regards',
 'drugs',
 ',',
 'sex',
 'violence',
 '.',
 'hardcore',
 ',',
 'classic',
 'use',
 'word.it',
 'called',
 'oz',
 'nickname',
 'given',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 '.',
 'focuses',
 'mainly',
 'emerald',
 'city',
 ',',
 'experimental',
 'section',
 'prison',
 'cells',
 'glass',
 'fronts',
 'face',
 'inwards',
 ',',
 'privacy',
 'high',
 'agenda',
 '.',
 'em',
 'city',
 'home',
 'many',
 '..',
 'aryans',
 ',',
 'muslims',
 ',',
 'gangstas',
 ',',
 'latinos',
 ',',
 'christians',
 ',',
 'italians',
 ',',
 'irish',
 '....',
 'scuffles',
 ',',
 'death',
 'stares',
 ','

In [30]:
#Tessting the function
text = "The cats are ruuning"
stemmed_text = stemmer(text.split())
print(stemmed_text)

the cat are ruun


In [31]:
%%time
df['review'] = df['review'].apply(stemmer)

CPU times: total: 2min 22s
Wall time: 3min 7s


In [32]:
df.review[0]

"one review mention watch 1 oz episod ll hook . right , exact happen me.th first thing struck oz brutal unflinch scene violenc , set right word go . trust , show faint heart timid . show pull punch regard drug , sex violenc . hardcor , classic use word.it call oz nicknam given oswald maximum secur state penitentari . focus main emerald citi , experiment section prison cell glass front face inward , privaci high agenda . em citi home mani .. aryan , muslim , gangsta , latino , christian , italian , irish .... scuffl , death stare , dodgi deal shadi agreement never far away.i would say main appeal show due fact goe show would n't dare . forget pretti pictur paint mainstream audienc , forget charm , forget romanc ... oz n't mess around . first episod ever saw struck nasti surreal , could n't say readi , watch , develop tast oz , got accustom high level graphic violenc . violenc , injustic ( crook guard ll sold nickel , inmat ll kill order get away , well manner , middl class inmat turn pr

Data Preprocessing

In [33]:
#Increasethe max_colwidth value to avoid truncation
pd.set_option('display.max_colwidth', 120)

In [34]:
#Extract the review text (input)
x = np.array(df.iloc[:, 0].values)

In [35]:
#Extract the semtiment(output)
y = np.array(df.sentiment.values)

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)


In [37]:
''' ALL machine learning are reproduction of human learning'''


' ALL machine learning are reproduction of human learning'

In [38]:
type(x_train)

numpy.ndarray

In [39]:
#Create a vectorizer (it will convert the text data into numerical data)
vectorizer = CountVectorizer(max_features = 1000) 

In [40]:
#Fit the vectorizer and tranform it with training data
x_train_final = vectorizer.fit_transform(x_train).toarray()

In [41]:
#Only transform the tezt data
x_test_final = vectorizer.transform(x_test).toarray()

In [42]:
print("x_train_final", x_train_final.shape)
print("y_train", y_train.shape)

x_train_final (40000, 1000)
y_train (40000,)


In [43]:
print(x_train_final)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Creation of Machine Learning Models

Model 1 = GaussianNB

In [52]:
#Create the model
model_v1 = GaussianNB()

In [53]:
#Train the model
model_v1.fit(x_train_final, y_train)

GaussianNB()

MultinomailNB

In [54]:
model_v2 = MultinomialNB(alpha = 1.0, fit_prior = True)

In [55]:
#Train the model
model_v2.fit(x_train_final, y_train)


MultinomialNB()

BernoulliNB

In [56]:
#Create the model
model_v3 = BernoulliNB(alpha = 1.0, fit_prior=True)

In [57]:
#Train the model
model_v3.fit(x_train_final, y_train)

BernoulliNB()

Evaluation, Interpretation, and Comparison of the Models

In [58]:
#Prediction with the test data of v1,v2,v3
ypred_v1 = model_v1.predict(x_test_final)
ypred_v2 = model_v2.predict(x_test_final)
ypred_v3 = model_v3.predict(x_test_final)

In [59]:
#Printing the accuracy of each model
print("Accuracy of GauusianNB Model = ", accuracy_score(y_test, ypred_v1) * 100)
print("Accuracy of MultinomialNB Model = ", accuracy_score(y_test, ypred_v2) * 100)
print("Accuracy of BernoulliNB Model = ", accuracy_score(y_test, ypred_v3) * 100)

Accuracy of GauusianNB Model =  78.59
Accuracy of MultinomialNB Model =  82.39999999999999
Accuracy of BernoulliNB Model =  82.92


In [60]:
#Import
from sklearn.metrics import roc_auc_score


In [61]:
#AUC(Area under Curve) of GaussianNb
y_proba = model_v1.predict_proba(x_test_final)[:, 1]
auc = roc_auc_score(y_test, y_proba)
print("Auc of GaussianNB = ", auc)

Auc of GaussianNB =  0.8576639855352913


In [62]:
#AUC(Area under Curve) of MultinomailNb
y_proba = model_v2.predict_proba(x_test_final)[:, 1]
auc = roc_auc_score(y_test, y_proba)
print("Auc of MultinomailNB = ", auc)

Auc of MultinomailNB =  0.8967627013723672


In [63]:
#AUC(Area under Curve) of BrernoulliNB
y_proba = model_v3.predict_proba(x_test_final)[:, 1]
auc = roc_auc_score(y_test, y_proba)
print("Auc of BrernoulliNB = ", auc)

Auc of BrernoulliNB =  0.9053642828498596


In [64]:
#62 Save the best model to disk
with open('model_v3.pkl', 'wb') as file:
    pickle.dump(model_v3, file)

In [65]:
#63 Load the model from disk
with open('model_v3.pkl', 'rb') as file:
    final_model = pickle.load(file)

In [66]:
#64 User review text (this text has a positive sentiment)
review_text = """This is probably the fastest-paced and most action-packed of the German Edgar Wallace "krimi"
series, a cross between the Dr. Mabuse films of yore and 60's pop thrillers like Batman and the Man
from UNCLE. It reintroduces the outrageous villain from an earlier film who dons a stylish monk's habit and
breaks the necks of victims with the curl of a deadly whip. Set at a posh girls' school filled with lecherous
middle-aged professors, and with the cops fondling their hot-to-trot secretaries at every opportunity, it
certainly is a throwback to those wonderfully politically-incorrect times. There's a definite link to a later
Wallace-based film, the excellent giallo "Whatever Happened to Solange?", which also concerns female students
being corrupted by (and corrupting?) their elders. Quite appropriate to the monk theme, the master-mind villain
uses booby-trapped bibles here to deal some of the death blows, and also maintains a reptile-replete dungeon
to amuse his captive audiences. <br /><br />Alfred Vohrer was always the most playful and visually flamboyant
of the series directors, and here the lurid colour cinematography is the real star of the show. The Monk appears
in a raving scarlet cowl and robe, tastefully setting off the lustrous white whip, while appearing against
purplish-night backgrounds. There's also a voyeur-friendly turquoise swimming pool which looks great both
as a glowing milieu for the nubile students and as a shadowy backdrop for one of the murder scenes.
The trademark "kicker" of hiding the "Ende" card somewhere in the set of the last scene is also quite
memorable here. And there's a fine brassy and twangy score for retro-music fans.<br /><br />Fans of the series
will definitely miss the flippant Eddie Arent character in these later films. Instead, the chief inspector
Sir John takes on the role of buffoon, convinced that he has mastered criminal psychology after taking a few
night courses. Unfortunately, Klaus Kinski had also gone on to bigger and better things. The krimis had
lost some of their offbeat subversive charm by this point, and now worked on a much more blatant pop-culture
level, which will make this one quite accessible to uninitiated viewers."""

In [67]:
#Data transformation flow
task1= clean_data(review_text)
task2 = clean_special_charecters(task1)
task3 = convert_to_lowercase(task2)
task4 = remove_stopwords(task3)
task5 = stemmer(task4)

In [68]:
#Data transformation flow
task1= clean_data(review_text)
task2 = clean_special_charecters(task1)
task3 = convert_to_lowercase(task2)
task4 = remove_stopwords(task3)
task5 = stemmer(task4)

In [69]:
if not review_text:
    raise ValueError("Input text is empty.")


In [70]:
if not isinstance(review_text, str):
    raise TypeError("Input must be a string.")


In [71]:
print(task5)

thisisprobablythefastestpacedandmostactionpackedofthegermanedgarwallacekrimiseriesacrossbetweenthedrmabusefilmsofyoreand60spopthrillerslikebatmanandthemanfromuncleitreintroducestheoutrageousvillainfromanearlierfilmwhodonsastylishmonkshabitandbreaksthenecksofvictimswiththecurlofadeadlywhipsetataposhgirlsschoolfilledwithlecherousmiddleagedprofessorsandwiththecopsfondlingtheirhottotrotsecretariesateveryopportunityitcertainlyisathrowbacktothosewonderfullypoliticallyincorrecttimestheresadefinitelinktoalaterwallacebasedfilmtheexcellentgiallowhateverhappenedtosolangewhichalsoconcernsfemalestudentsbeingcorruptedbyandcorruptingtheireldersquiteappropriatetothemonkthemethemastermindvillainusesboobytrappedbiblesheretodealsomeofthedeathblowsandalsomaintainsareptilerepletedungeontoamusehiscaptiveaudiencesalfredvohrerwasalwaysthemostplayfulandvisuallyflamboyantoftheseriesdirectorsandheretheluridcolourcinematographyistherealstaroftheshowthemonkappearsinaravingscarletcowlandrobetastefullysettingoffthel

In [72]:
print("Step 1 (Clean Data):", task1)
print("Step 2 (Clean Special Characters):", task2)
print("Step 3 (Convert to Lowercase):", task3)
print("Step 4 (Remove Stopwords):", task4)
print("Step 5 (Stemmed Text):", task5)


Step 1 (Clean Data): This is probably the fastest-paced and most action-packed of the German Edgar Wallace "krimi"
series, a cross between the Dr. Mabuse films of yore and 60's pop thrillers like Batman and the Man
from UNCLE. It reintroduces the outrageous villain from an earlier film who dons a stylish monk's habit and
breaks the necks of victims with the curl of a deadly whip. Set at a posh girls' school filled with lecherous
middle-aged professors, and with the cops fondling their hot-to-trot secretaries at every opportunity, it
certainly is a throwback to those wonderfully politically-incorrect times. There's a definite link to a later
Wallace-based film, the excellent giallo "Whatever Happened to Solange?", which also concerns female students
being corrupted by (and corrupting?) their elders. Quite appropriate to the monk theme, the master-mind villain
uses booby-trapped bibles here to deal some of the death blows, and also maintains a reptile-replete dungeon
to amuse his captive

In [73]:
type(task5)

str

In [74]:
task5_array = np.array(task5)

In [75]:
type(task5_array)

numpy.ndarray

In [76]:
#Apply the vectorizer with another conversion to NUmpy array to adjust the shape from 0-d to 1-d
final_review = vectorizer.transform(np.array([task5_array])).toarray()

In [77]:
type(final_review)

numpy.ndarray

In [78]:
prediction = final_model.predict(final_review.reshape(1, 1000))

In [79]:
print(prediction)

[1]


In [80]:
#Condititon stracuture to check the value of prediction
if prediction == 1:
    print("The text Indicates Postive Sentiment")
else:
    print("The text Indicates Negative Sentiment")


The text Indicates Postive Sentiment
